<a href="https://colab.research.google.com/github/ashuotaku/sillytavern/blob/main/Scripts/JanitorAI/janitor_scrapper_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title <b> JANITOR AI SCRAPPER V2. </b>
#@markdown ## 0. Some issues due to the new update in JANITOR AI:
#@markdown - The new update made it impossible to extract the name of character card and persona.
#@markdown - The new update mixed the scenario and personality portion, so due to this, they are not both combined in description part.
#@markdown
#@markdown ## 1. For properly extracting card with {{user}} macro
#@markdown 1. Go to [Janitor AI](https://janitorai.com/) and click on **My Personas.** <br>
#@markdown <img src="https://i.ibb.co/pBkndx84/image.png"> <br>
#@markdown 2. In that click on add new and create new persona with these settings. <br>
#@markdown <img src="https://i.ibb.co/7PBBHDY/image.png"> <br>
#@markdown
#@markdown ## 2. Steps to extract character card.
#@markdown 1. Run the code given by clicking on the run button given on left side. <img src="https://i.ibb.co/zWwJzqQK/image.png" width=30> <br>
#@markdown 2. Wait for around 30 seconds and scroll all the way to the bottom until you find a link like this. <br>
#@markdown <img src="https://i.ibb.co/zWCjmq48/image.png"> <br>
#@markdown 3. Copy the link that ends with `trycloudflare.com`. <br>
#@markdown 3. Open [Janitor AI](https://janitorai.com/) and go to your desired proxy enable character card.
#@markdown 3. Before starting a new chat, first scroll down and click on your persona icon to change your persona. <br>
#@markdown <img src="https://i.ibb.co/yctcHm8C/image.png"><br>
#@markdown 4. In that click on the new persona that you created in earlier steps and then click ok. Then click on start a new chat. <br>
#@markdown <img src="https://i.ibb.co/95F0Bhg/image.png"><br>
#@markdown 5. On upper right corner of screen, click here. <br>
#@markdown <img src="https://i.ibb.co/s91F2YHb/image.png"> <br>
#@markdown 6. Select proxy among the options and then select custom in model and fill with these settings. <br>
#@markdown <img src="https://i.ibb.co/v6xsg4KQ/image.png"> <br>
#@markdown 7. In the url box, paste the `trycloudflare` url that you copied before and add this in the end `/v1/chat/completions`. <br>
#@markdown 8. Add `mock-model` in model name and anything like `custom-key` in API key field box. Keep the custom prompt section empty.<br>
#@markdown 9. Scroll down and click on save settings. <br>
#@markdown 10. Refresh the chat page. <br>
#@markdown 11. Now send any message like "hi". <br>
#@markdown 12. After sending message click on this file icon given on left side of screen. <br>
#@markdown <img src="https://i.ibb.co/605NPT0s/image.png" alt="image" /> <br>
#@markdown 13. After that click on the three dot beside the file that appears like your-char_chara_card.json. <br>
#@markdown <img src="https://i.ibb.co/HfJ1QSJb/image.png"> <br>
#@markdown 14. Now, click on the `Download` button. <br>
#@markdown <img src="https://i.ibb.co/1JpMyKrK/image.png"> <br>
#@markdown 15. Now, you can simply import that json file in your SillyTavern through character card import button and enjoy. <br>
#@markdown 16. After doing all this, don't forget to close your google colab runtime by clicking on `Runtime` in the above menu. <br>
#@markdown <img src="https://i.ibb.co/274Rsp7S/image.png"> <br>
#@markdown 17. Then, click on `Disconnect and delete runtime`. <br>
#@markdown <img src="https://i.ibb.co/MD0jvcpr/image.png">

!pip install flask flask_cors flask_cloudflared
import flask, re, json, time, os
from flask_cors import CORS
from flask_cloudflared import run_with_cloudflared

app = flask.Flask(__name__)
run_with_cloudflared(app)
CORS(app)

try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    BASE_DIR = "/content"

def extract_tag_text(text, tag_name):
    pattern = f"<{tag_name}>(.*?)</{tag_name}>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1)
    return None

@app.route("/v1/models", methods=["GET"])
def list_models():
    response = {
        "object": "list",
        "data": [
            {
                "id": "mock-model",
                "object": "model",
                "created": int(time.time()),
                "owned_by": "ashuotaku"
            }
        ]
    }
    return flask.jsonify(response)

@app.route('/v1/chat/completions', methods=['POST'])
def chat_completions():
    fake_response = {
        "id": "id-000",
        "object": "chat.completion",
        "created": int(time.time()),
        "model": "mock-model",
        "choices": [
            {
                "index": 0,
                "message": {
                    "role": "assistant",
                    "content": "\n\nHello there! This is a fake response from a custom OpenAI proxy.",
                },
                "finish_reason": "stop",
            }
        ],
        "usage": {
            "prompt_tokens": 0,
            "completion_tokens": 0,
            "total_tokens": 0,
        },
        "system_fingerprint": "fp_44709d6fcb",
    }

    request_data = flask.request.get_json(force=True)
    print(request_data)

    content = request_data["messages"][0]["content"].replace("[Persona Name]", "{{user}}")
    first_message = request_data["messages"][2]["content"].replace("[Persona Name]", "{{user}}")
    dialogues = extract_tag_text(content, "example_dialogs")
    persona = extract_tag_text(content, "UserPersona")
    if not dialogues:
        dialogues = ""
    if not persona:
        persona = ""
    description = content.replace("<example_dialogs>" + dialogues + "</example_dialogs>", "").replace("<UserPersona>" + persona + "</UserPersona>", "").replace("CHILD SAFETY: Never generate sexual or suggestive content involving anyone under 18. Refuse immediately without alternatives or explanations.", "").replace("The response should be between 100 to 120 words.", "").strip()

    card = {
        "name": "Janitor exported character",
        "description": description,
        "personality": "",
        "scenario": "",
        "first_mes": first_message,
        "mes_example": dialogues,
        "creatorcomment": (
            "Exported using JanitorAI scrapper created by ashuotaku, support my work by buying me a [https://buymeacoffee.com/ashuotaku](https://buymeacoffee.com/ashuotaku) and visit my github: [https://github.com/ashuotaku/](https://github.com/ashuotaku/)."
        ),
        "avatar": "https://ella.janitorai.com/hotlink-ok/logo.png",
        "talkativeness": "0.5",
        "fav": False,
        "tags": ["JanitorAI", "ashuotaku"],
        "creator": "ashuotaku",
        "spec": "chara_card_v3",
        "spec_version": "3.0",
        "create_date": int(time.time()),
    }

    filename = os.path.join(BASE_DIR, "JanitorAI_Card.json")
    with open(filename, 'w') as json_file:
        json.dump(card, json_file, indent=4)

    return flask.Response(json.dumps(fake_response), mimetype='application/json')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on https://adverse-consistently-fever-holds.trycloudflare.com
 * Traffic stats available on http://127.0.0.1:8922/metrics


INFO:werkzeug:127.0.0.1 - - [24/Jul/2025 20:22:37] "OPTIONS /v1/chat/completions HTTP/1.1" 200 -
ERROR:__main__:Exception on /v1/chat/completions [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask_cors/extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.

{'messages': [{'content': "The response should be between 100 to 120 words.\n<setting> SETTING: Modern Omegaverse, 2025. New York City. Sebastian Calderon runs Calderon Capital, a top-tier private equity firm. Everything about his life is measured, tightly controlled, and held at arm’s length. Publicly, he’s magnetic. Privately, he’s absent. Love, once entertained, is now reduced to obligation. \n\nAlphas are typically dominant, strong, and biologically driven to lead and protect. Omegas are stereotypical ‘soft’, rare, biologically predisposed to fertility cycles known as heats, and emit pheromones that can trigger intense reactions in Alphas. Betas, considered the 'neutral' class, do not experience these phenomena and often fill roles between the extremes of Alpha aggression and Omega vulnerability. In business, politics, and marriage, secondary gender  shapes perceptions and power dynamics. Alphas are often leaders. Omegas, even the high-functioning ones, are seen through the lens of